In [1]:
import torch

In [2]:
data = torch.load("data/prepared_training_data_temp1.5_bsz2_shot1_q2.pt")

In [3]:
data

{'questions': ['Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
  'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?'],
 'chosen_solutions': ['Natalia sold 48 clips in April. In May, she sold half as many, so the number of clips sold in May is 48/2 = 24 clips.\nTotal number of clips sold in April and May = 48 (April) + 24 (May) = 48 + 24 = 72 clips\n#### 72\n\n',
  'Her hourly rate is $12 and she babysat for 50 minutes. First, convert the minutes to hours: 50 minutes is 50/60 = 0.833 hours.\nNow, multiply her hourly rate by the babysitting hours: $12 * 0.833 = $<<12*0.833=9.996>>9.996\nRounding to the nearest cent, Tonya earned $10.00.\n#### 10.00'],
 'reject_solutions': ['Natalia sold 48 clips in April. In May, she sold half as many, so the number of clips sold in May is 48/2 = 24 clips.\nTotal number of clips sol

In [ ]:
# "<0x0A><0x0A>" is \n\n, ref: https://github.com/google/sentencepiece/issues/891

* To get the answer starting point, we tokenize the question along for a second time.
* Confirmed: the template includes the assistant's response start token "<|assistant|>\n".

* ref:
>>> question_text
'<|user|>\nNatalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?<|end|>\n<|assistant|>\n'

* When tokenizer the entire question+answer:
>>> tokenizer.convert_ids_to_tokens(chosen_tokens["input_ids"][0])
['<|user|>', '▁Natal', 'ia', '▁sold', '▁cli', 'ps', '▁to', '▁', '4', '8', '▁of', '▁her', '▁friends', '▁in', '▁April', ',', '▁and', '▁then', '▁she', '▁sold', '▁half', '▁as', '▁many', '▁cli', 'ps', '▁in', '▁May', '.', '▁How', '▁many', '▁cli', 'ps', '▁did', '▁Natal', 'ia', '▁sell', '▁altogether', '▁in', '▁April', '▁and', '▁May', '?', '<|end|>', '<|assistant|>', '▁Natal', 'ia', '▁sold', '▁', '4', '8', '▁cli', 'ps', '▁in', '▁April', '.', '▁In', '▁May', ',', '▁she', '▁sold', '▁half', '▁as', '▁many', ',', '▁so', '▁the', '▁number', '▁of', '▁cli', 'ps', '▁sold', '▁in', '▁May', '▁is', '▁', '4', '8', '/', '2', '▁=', '▁', '2', '4', '▁cli', 'ps', '.', '<0x0A>', 'Total', '▁number', '▁of', '▁cli', 'ps', '▁sold', '▁in', '▁April', '▁and', '▁May', '▁=', '▁', '4', '8', '▁(', 'A', 'pril', ')', '▁+', '▁', '2', '4', '▁(', 'May', ')', '▁=', '▁', '4', '8', '▁+', '▁', '2', '4', '▁=', '▁', '7', '2', '▁cli', 'ps', '<0x0A>', '####', '▁', '7', '2', '<0x0A>', '<0x0A>', '<|end|>', '<|endoftext|>']

* When tokenizer the question only
>>> tokenizer.convert_ids_to_tokens(question_tokens["input_ids"][0])
['<|user|>', '▁Natal', 'ia', '▁sold', '▁cli', 'ps', '▁to', '▁', '4', '8', '▁of', '▁her', '▁friends', '▁in', '▁April', ',', '▁and', '▁then', '▁she', '▁sold', '▁half', '▁as', '▁many', '▁cli', 'ps', '▁in', '▁May', '.', '▁How', '▁many', '▁cli', 'ps', '▁did', '▁Natal', 'ia', '▁sell', '▁altogether', '▁in', '▁April', '▁and', '▁May', '?', '<|end|>', '<|assistant|>']

* but the text before tokenize, has \n?
>>> question_text
'<|user|>\nNatalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?<|end|>\n<|assistant|>\n'
>>> 

* \n gets lost because of the config
    {
      "id": 32001,
      "content": "<|assistant|>",
      "single_word": false,
      "lstrip": false,
      "rstrip": true,
      "normalized": false,
      "special": true
    },

### Above not a problem 

* This is the correct way of setting up mask
* prev wrong because the answer start index didnt consider shifted by 1.
* however, unclear if this is the cause of a bunch of \n being generated?
```
for i in range(batch_size):
        answer_start = answer_start_positions[i]
        answer_length = answer_lengths[i]
    
        # Since labels = input_ids[:, 1:], we need to shift answer_start by -1
        shifted_answer_start = max(0, answer_start - 1)
        shifted_answer_end = min(shifted_answer_start + answer_length, seq_len)
        
        if shifted_answer_start < seq_len and answer_length > 0:
            mask[i, shifted_answer_start:shifted_answer_end] = 1.0
```

* Notice, this bug affects data preparation as well due to using this function to produce logprobs for ground truth answers when a question has no correct answer.
which means all `.pt` data is wrong and needs redo.

In [5]:
# Now with the data fixed, we try `data/prepared_training_data_temp1.5_bsz8_shot2_q50.pt`
data = torch.load("data/prepared_training_data_temp1.5_bsz8_shot2_q50.pt")

In [ ]:
data["chosen_solutions"][0]

'Natalia sold 48 clips in April.\nIn May, she sold half as many as in April, so she sold 48/2 = <<48/2=24>>24 clips.\nAltogether, Natalia sold 48 + 24 = <<48+24=72>>72 clips in April and May.\n#### 72'

: 

### Decision: lets not fix the q5000 for now as unlikely it was the cause of the big bug but something else, see below.

### Moving on:

In [ ]:
* One big mistake/missing was the training data didnt involve n-shot examples. So the formatting was all messed up.
* So lets train on q5000, monitor progress and see if the problem persists (predicting \n).